<a href="https://colab.research.google.com/github/vnwrywn/CryptoLearning/blob/main/RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kriptosistem Rivest–Shamir–Adleman (RSA)

Implementasi Kriptosistem Rivest–Shamir–Adleman (RSA) dengan python.

Oleh Muhammad Ivan Wiryawan (NIM 195150200111060 & No. Absen 6)

Dibuat dalam rangka pengerjaan tugas mata kuliah Kriptografi.

---

## Import Module

Berbagai modul diperlukan untuk mengimplementasikan RSA dengan python. Selain itu beberapa modul juga diperlukan untuk mendemonstrasikan penyerangan terhadap implementasi RSA yang tidak aman.

In [ ]:
from math import gcd, sqrt, ceil, floor
from random import randrange
import decimal
import sys

## Fungsi Invers Pengalian Modulo

Untuk menentukan nilai d, kita perlu melakukan invers pengalian modulo terhadap $e$ modulo $\lambda$. Hal ini dapat dilakukan dengan mudah pada python 3.8+. Akan tetapi, karena gogole colab menggunakan python versi 3.7.15 kita perlu membuat fungsi tersendiri untuk menerapkan invers pengalian modulo. Untuk menerapkan fungsi ini, kita memerluka fungsi FPB yang efisien dan dirancang khusus untuk fungsi invers pengalian modulo kita.

In [ ]:
def egcd(a, b):

  if a == 0:
    return (b, 0, 1)

  else:
    g, y, x = egcd(b % a, a)
    return (g, x - (b // a) * y, y)

def mod_inverse(a, m):
  g, x, y = egcd(a, m)

  if g == 1:
    return x % m

  else:
    raise Exception('no modular inverse')

## Generasi Key

Kriptosistem RSA menggunakan beberapa kunci, yang di antaranya adalah: n (modulus), e (eksponen public), dan d (eksponen rahasia). Variabel n diperoleh dengan mengalikan p dan q (bilangan prima). Variabel $\lambda(n)$ perlu dikomputasikan dengan menggunakan rumus $\lambda(n) = lcm(lambda(p), lambda(p)) = lcm(p-1, q-1)$. Fungsi untuk menentukan KPK tersedia dalam library math untuk python 3.5+, akan tetapi tidak dapat di-_import_ melalui _instance_ colab ini, sehingga perlu dibuat fungsi-nya tersendiri dengan menerapkan rumus $lcm(a, b) = \dfrac{\left | ab \right |} {gcd(a,b)}$. Variabel $\lambda(n)$ digunakan dalam menentukan $e$, sehingga $1 < e < \lambda(n)$ dan $gcd(e, \lambda(n))$. Variabel $d$ kemudian dikomputasikan dengan $d \equiv e^{−1} (\mod \lambda(n))$.

In [ ]:
# Gunakan math.lcm() pada python 3.9+
def lcm(a, b):
  return abs(a * b) // gcd(a, b)

In [ ]:
def generate_keys(p, q, e=0):
  n = p * q
  cf_n = lcm((p - 1), (q - 1))
  
  if e == 0:
    e = randrange(1, cf_n)
    g = gcd(e, cf_n)

    while True:
      e = randrange(1, cf_n)
      g = gcd(e, cf_n)
      d = mod_inverse(e, cf_n)

      if g == 1 and e != d:
        break

  else:
    d = mod_inverse(e, cf_n)

  return ((e, n), (d, n))

## Fungsi Enkripsi

Untuk melakukan enkripsi terhadap suatu plaintext $m$ menjadi ciphertext $c$, digunakan rumus berikut. $$c \equiv m^{e} (\mod n)$$ Untuk melakukan enkripsi, plaintext pertama-tama harus di-encode ke dalam bentuk integer agar dapat dioperasikan. Untuk mengubah plaintext string menjadi integer, setiap huruf pada string dapat di-_decode_ sesuai dengan nomor ASCII-nya dan kemudian diubah menjadi string heksadesimal. String heksadesimal ini kemudian di-_cast_ menjadi integer secara keseluruhannya dan diubah dengan mengikuti rumus di atas.

In [ ]:
def encrypt(p, key):
  e, n = key
  m = '0x'

  for char in p:
    m += hex(ord(char))[2:]

  return pow(int(m, 0), e, n)

## Fungsi Dekripsi

Untuk melakukan dekripsi terhadap suatu ciphertext $c$ menjadi plaintext $m$, digunakan rumus berikut. $$m \equiv c^{d} (\mod n)$$ Untuk melakukan enkripsi, plaintext pertama-tama diubah dengan mengikuti rumus di atas, lalu diubah menjadi string heksadesimal. String heksadesimal ini kemudian ditranslasikan secara langsung menjadi string ASCII.

In [ ]:
def decrypt(c, key):
  d, n = key
  res = bytes.fromhex(hex(pow(c, d, n))[2:]).decode('utf-8')
  
  return res

Catatan: $d$ dapat digunakan untuk melakukan _signing_ dengan mengenkripsi hash dari pesan dengan menggunakan rumus $c \equiv hash(m)^{d} (\mod n)$. Verifikasi _sign_ kemudian dilakukan dengan melakukan dekripsi dengan menggunakan rumus $hash(m) \equiv c^{e} (\mod n)$. Proses _signing_ dapat digunakan untuk memastikan bahwa suatu pesan benar-benar dikirim oleh seseorang, karena $d$ (harusnya) hanya diketahui oleh pengirim.

## Contoh

Untuk mendemonstrasikan proses enkripsi dan dekripsi RSA, pertama-tama kita perlu menentukan nilai p, q, dan e (dapat ditentukan secara otomatis melalui fungsi `generate_keys()`). Kemudian kita mengenerasikan kunci public dan private dengan menggunakan fungsi `generate_keys()`.

In [ ]:
p = 30015612851481800163145824211828338261413382292790584457054499062507044817040728341690870065470432181409810774142161519778252587630533918037805771246208706720205699941638685693936864749777451034676171694531262268574649020866993235775129431727651701029771811813255651429786267886419653775125853176632392525213
q = 17719842803345829609147997585534573574103070131643374729678771438471674919388468936368916163398796348115998882101971455756733471133506792568528967349802844671228181682201529458832662110189178035470305127209240816034739235935127061585044161495037790251195536985856822323336983333522129818151265418420956257327
e = 65537
public, private = generate_keys(p, q, e)
print('e:', public[0])
print('n:', public[1])
print('d:', private[0])
print('n:', private[1])

e: 65537
n: 531871941374344372167169323250866207346623640488167774883925228871921220128151440279892448899228603322466710861747228131558066199714472637055546381930534204457125050708383810101794235938758578884734881281418698033490027180180153617647402022407743034218773865326123331835947057340209042659146567448074386113277175774290160285702889707227056592283842009094687967526430087455854433095071949209724515885774729123181618637569563216296254704443034750154450376184667529745699446628606332688885791287435112772451055843557844644448275344991264065190788445033398674763045313878903833635674603665076430533305795161234963485651
d: 117002575320718415758031037937466440504085829759035580061668218329287093254002461426601910917194543145093650464528583669876140812079947998968976489437913112068882796833281495799892694196088963971210503737342468674929058727995747054421511435632580547237927625866407068908843076821853209149288433448264162604245370417836727206014181315596379049865320499115233410718119673

Setelah key digenerasikan, kita dapat melakukan proses enkripsi dengan menggunakan fungsi `encrypt()`. Ciphertext yang telah diperoleh kemudian dapat didekripsi dengan menggunakan fungsi `decrypt()`.

In [ ]:
plaintext = 'lorem ipsum'
ciphertext = encrypt(plaintext, public)
print('Ciphertext:', ciphertext)
decrypted_ciphertext = decrypt(ciphertext, private)
print('Decrypted Ciphertext:', decrypted_ciphertext)

Ciphertext: 54611987937732336924366393677214644087876697644416474932828921817883196165901396665540422061897789021075314241821651508361361151090659532911063444624936810773493991034303895525417807795259835431167467294597647357166385189599101656199737930288282415959282288216150219638878881828164858187890851791748170039040475978631828431484244183532564856712148816336414241899602719018367134883593665206535400383471062192404924459108234836458966650800657801195524008096661715691061350652744718141394284206822990363214954276098048027565489000903061420842243140992858129671368239717404559357109009486817446977775301985594808563416
Decrypted Ciphertext: lorem ipsum


## Pecobaan Penyerangan

### Penyerangan dengan Metode Brute Force

Solusi penyerangan yang paling simpel dan mahal adalah dengan melakukan serangan bruteforce untuk mencari variabel p atau q dengan mencari faktor n secara sekuensial. Hal ini sangat mudah jika p atau q bernilai sangat kecil. Akan tetapi, metode ini dapat menjadi sangat mahal ataupun tidak mungkin untuk dilakukan apabila ukuran p dan q sangat besar, seperti sebagaimana lazimnya.

In [ ]:
def bruteforce(n):
  for i in range(2, ceil(sqrt(n))):
    if n % i == 0:
      return i

In [ ]:
p = 61
q = 53
e = 17
plaintext = 'A'
public, private = generate_keys(p, q, e)
print('e:', public[0])
print('n:', public[1])
ciphertext = encrypt(plaintext, public)
print('Ciphertext:', ciphertext)

e: 17
n: 3233
Ciphertext: 2790


In [ ]:
n = 3233
p_cracked = bruteforce(n)
q_cracked = n // p_cracked
d_cracked = mod_inverse(17, lcm((p_cracked - 1), (q_cracked - 1)))
decrypt(2790, (d_cracked, n))

'A'

Dapat dilihat bahwa proses penyerangan dapat dilakukan dalam waktu yang sangat cepat, dikarenakan ukutan p dan q yang sangat kecil.

Untuk percobaan berikutnya, kita akan melakukan penyerangan dengan metode brute force terhadap nilai p dan q yang sangat berjauhan, sehingga salah satunya menjadi sangat kecil.

In [ ]:
p = 2455782092362068153759284200389162744625754916643583857406201854084923116558938934679657691870353654367809190558995974904050863664254954066259836164398999
q = 53819
e = 65537
plaintext = 'A'
public, private = generate_keys(p, q, e)
print('e:', public[0])
print('n:', public[1])
ciphertext = encrypt(plaintext, public)
print('Ciphertext:', ciphertext)

e: 65537
n: 132167736428834145967170916380744349753013503858841039621744377584996477210085534525524497318770563324421122826694604373361113431546537372892038122531789727181
Ciphertext: 95389788262796403535021499242066002208896350183600540268800980053697058561013208668958347077268527507690201575592282512337498695579004598618013511329704297325


In [ ]:
n = 132167736428834145967170916380744349753013503858841039621744377584996477210085534525524497318770563324421122826694604373361113431546537372892038122531789727181
p_cracked = bruteforce(n)
q_cracked = n // p_cracked
d_cracked = mod_inverse(65537, lcm((p_cracked - 1), (q_cracked - 1)))
decrypt(95389788262796403535021499242066002208896350183600540268800980053697058561013208668958347077268527507690201575592282512337498695579004598618013511329704297325, (d_cracked, n))

'A'

Dapat dilihat bahwa, meskipun nilai $n$ cukup besar, salah satu faktornya dapat dipecahkan dengan mudah karena berukuran sangat kecil. Maka dari itu, kita harus menghindari penggunaan nilai p dan q yang terlalu kecil. Sejak tahun 2020, rekomendasi ukuran key minimum adalah 2048 bit.

### Penyerangan dengan Metode Brute Force Mundur

Dalam kondisi selisih nilai p dan q sangat kecil, penyerangan brute-force dapat dilakukan dari akar kuadrat nilai n dan kemudian di-dekremen.

In [ ]:
p = 14067596681028047262628791508362831627045477834885788430404510354806103106077
q = 14067596681028047262628791508362831627045477834885788430404510354806103106173
e = 65537
plaintext = 'lorem ipsum'
public, private = generate_keys(p, q, e)
print('e:', public[0])
print('n:', public[1])
ciphertext = encrypt(plaintext, public)
print('Ciphertext:', ciphertext)

e: 65537
n: 197897276380071330918336632165112876285525821472597233236725533580110473596837775424512081154431671785022967033612984943230533689788048854204373012513321
Ciphertext: 128089455737117345298530888745317906967266402716600279625902361067424552328613473394941009416027849713950912143855435454617838648724785301935135541356005


In [ ]:
def rev_bruteforce(n):
  decimal.getcontext().prec = 1000
  for i in range(ceil(decimal.Decimal(n).sqrt()), 2, -1):
    if n % i == 0:
      return i

In [ ]:
n = 197897276380071330918336632165112876285525821472597233236725533580110473596837775424512081154431671785022967033612984943230533689788048854204373012513321
p_cracked = rev_bruteforce(n)
q_cracked = n // p_cracked
d_cracked = mod_inverse(65537, lcm((p_cracked - 1), (q_cracked - 1)))
decrypt(128089455737117345298530888745317906967266402716600279625902361067424552328613473394941009416027849713950912143855435454617838648724785301935135541356005, (d_cracked, n))

'lorem ipsum'

### Penyerangan dengan Faktorisasi Fermat

Dalam hal selisih p dan q terlalu jauh untuk penyerangan brute force mundur, tetapi masih terlalu dekat, dapat digunakan faktorisasi fermat untuk mencari faktor dari n.

In [ ]:
p = 14067596681028047262628791508362831627045477834885788430408510354806103106123
q = 14067596681028047262628791508362831627045477834885384610408510332604131196127
e = 65537
plaintext = 'lorem ipsum'
public, private = generate_keys(p, q, e)
print('e:', public[0])
print('n:', public[1])
ciphertext = encrypt(plaintext, public)
print('Ciphertext:', ciphertext)

e: 65537
n: 197897276380071330918336632165112876285525821472591552459946341295184716862957963430874528476670634156894790593742866435379931202427814195497778007585621
Ciphertext: 170374534219219694339500499655337169628519903947121941775857651910843570653507096122024552055029063982556791721701153878701093244706030888834262759271749


In [ ]:
def fermat_factor(n):
  decimal.getcontext().prec = 1500

  for a in range(ceil(decimal.Decimal(n).sqrt()), n):
    b1 = (decimal.Decimal(a) ** 2) - n
    b = round(b1.sqrt())

    if(b * b == b1):
      return (int(a-b), int(a + b))

  return (n, n)

In [ ]:
n = 197897276380071330918336632165112876285525821472591552459946341295184716862957963430874528476670634156894790593742866435379931202427814195497778007585621
p_cracked, q_cracked = fermat_factor(n)
d_cracked = mod_inverse(65537, lcm((p_cracked - 1), (q_cracked - 1)))
decrypt(170374534219219694339500499655337169628519903947121941775857651910843570653507096122024552055029063982556791721701153878701093244706030888834262759271749, (d_cracked, n))

'lorem ipsum'

Dapat dilihat bahwa, meskipun ukuran $n$ cukup besar, selisih antara $p$ dan $q$ yang relatif kecil menimbulkan celah keamanan yang cukup fatal. Maka dari itu, perlu diperhatikan agar selisih tersebut cukuplah besar.

### Penyerangan dengan Pengakaran Kubik

Dalam hal nilai $e$ yang digunakan terlalu kecil, seperti 3, besar plaintext cukup kecil, dan ukuran n cukup besar, kita dapat memecahkan plaintext dari ciphertext tanpa mencari nilai $d$ dan hanya dengan mengakar kubik-kan nilai $c$.

In [ ]:
p = 30015612851481800163145824211828338261413382292790584457054499062507044817040728341690870065470432181409810774142161519778252587630533918037805771246208706720205699941638685693936864749777451034676171694531262268574649020866993235775129431727651701029771811813255651429786267886419653775125853176632392525213
q = 17719842803345829609147997585534573574103070131643374729678771438471674919388468936368916163398796348115998882101971455756733471133506792568528967349802844671228181682201529458832662110189178035470305127209240816034739235935127061585044161495037790251195536985856822323336983333522129818151265418420956257327
e = 3
plaintext = 'test'
n = p * q
encrypt(plaintext, (e, n))
ciphertext = encrypt(plaintext, (e, n))
print('Ciphertext:', ciphertext)

Ciphertext: 7446927644895231780144668992


In [ ]:
def cube_root_dec(c):
  decimal.getcontext().prec = 1000
  m = floor(decimal.Decimal(c) ** (decimal.Decimal('1') / 3))
  return bytes.fromhex(hex(m)[2:]).decode('utf-8')

In [ ]:
cube_root_dec(ciphertext)

'tess'

Dapat dilihat bahwa hasil dekripsi tidaklah sempurna, di mana karakter terakhir tidak berhasil di-dekripsi dengan tepat. Hal ini dikarenakan sifat dari operasi pengakaran yang tidak dapat memberikan hasil yang sempurna.

## Kesimpulan

Keamanan dari kriptosistem RSA tersendiri berasal dari kesulitan permasalahan RSA, yakni menghitung salah satu bilangan prima (p) hanya dengan public key (N, e) dan sebuah ciphertext (c). Akan tetapi, kesulitan tersebut dapat dilangkahi apabila implementasi yang dilakukan tidak tepat. Hal ini telah didemonstrasikan pada percobaan-percobaan yang telah dilakukan. Meskipun tidak mencakup seluruh implementasi berbahaya, percobaan-percobaan ini mencakup berbagai implementasi yang paling mudah diserang. Maka dari itu, penting halnya untuk mengikuti standar-standar yang terpancang, seperti PKCS#1, dalam mengimplementasikan algoritma RSA.